<a href="https://colab.research.google.com/github/Steve-Falkovsky/Hypencoder-Entity-Linking/blob/main/notebooks/Hypencoder_Vs_fine_tuned_Hypencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import importlib.util

REPO_NAME = "Hypencoder-Entity-Linking"
GIT_URL = f"https://github.com/Steve-Falkovsky/{REPO_NAME}.git"
BRANCH_NAME = "Professional-Structure"

if not os.path.exists(REPO_NAME):
    !git clone -b {BRANCH_NAME} --single-branch {GIT_URL}

    # Move into the downloaded repo (The Root)
    os.chdir(REPO_NAME)


%pip install -q -e "./hypencoder-paper"

os.chdir("hypencoder-paper")

print(f"📍 Working Directory is now: {os.getcwd()}")
print("✅ Environment Ready!")

In [ ]:
from datasets import load_dataset

# there are all "positive" pairs"
dataset = load_dataset("Stevenf232/BC5CDR_MeSH2015_nameonly")

In [ ]:
train_pairs = dataset['train']
print(train_pairs)

mention_names = train_pairs['mention']
entity_names = train_pairs['entity']
print(mention_names[:3])
print(entity_names[:3])

### Load the model

In [ ]:
# Core Hypencoder model for outputing dense vector representations
from hypencoder_cb.modeling.hypencoder import Hypencoder, HypencoderDualEncoder, TextEncoder
from transformers import AutoTokenizer

model_name = "Stevenf232/SapBERT_freeze_hypencoder"

dual_encoder = HypencoderDualEncoder.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


query_encoder: Hypencoder = dual_encoder.query_encoder
passage_encoder: TextEncoder = dual_encoder.passage_encoder

### Move the model to the GPU

In [ ]:
import torch

# Setup the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")  # This should say 'cuda'

# Move the model to the GPU
passage_encoder.to(device)
query_encoder.to(device)

### Load datasets and tokenise

In [ ]:
# convert from type "datasets" to python list
queries = list(mention_names)
passages = list(entity_names)


# the output of the tokenizer contains 3 fields:
# input_ids, token_type_ids, and attention_mask
# all contain a tensor in the shape (number of queries, max number of tokens)

query_inputs = tokenizer(queries, return_tensors="pt", padding=True, truncation=True)
passage_inputs = tokenizer(passages, return_tensors="pt", padding=True, truncation=True)


In [ ]:
print(f"query_inputs:\n{query_inputs}")
print("\n\n\n")
print(f"passage_inputs:\n{passage_inputs}")

# Passage Encodings


In [ ]:
from tqdm import tqdm
import torch
from torch.amp import autocast

def batch_encode_passages(encoder ,passages):
  batch_size=256
  entity_name_features = []

  num_passages = passages["input_ids"].shape[0]

  with torch.no_grad(): # Disable gradient calculation (saves tons of memory)
    for i in tqdm(range(0, num_passages, batch_size), desc="Extracting features"):

        # extract entity features
        # Autocast does the math in fp16 where possible (default is fp32)
        # this will save memory and increase speed. The loss in precision shouldn't matter much (can check on a small sample if we want)
        with autocast("cuda"):
          features = encoder(
              input_ids=passages["input_ids"][i:i + batch_size].to(device),
              attention_mask=passages["attention_mask"][i:i + batch_size].to(device)
            ).representation

          entity_name_features.append(features.detach().cpu()) # Detach and move to CPU to save VRAM/RAM


  features_tensor = torch.cat(entity_name_features, dim=0)

  return features_tensor

In [ ]:
passage_embeddings = batch_encode_passages(passage_encoder, passage_inputs)

## Now, we create the q-nets.

For each q-net, we feed through it all the passages the calculate the similarity.

But the q-nets are created in batches, and every batch is represented as a single object `NoTorchSequential`. (Check out the `RepeatedDenseBlockConverter` class in q_net.py for more info)

This object expects an input in the shape (N, M, H):

* N = number of queries (mentions)

* M = number of passages (entities)

* H = Hidden dimension (e.g., 768 for BERT)



---



The passage embeddings have the shape (M, H) so we must create an additional dimension of size N.

This will be done like so:
`passages_batch = passages.unsqueeze(0).expand(num_queries, -1, -1)`

* `.unsqueeze()` adds a new dimension (in our case at location 0)

* `.expand()` "expands" that new dimension to be size "num_queries"

* `.expand()` creates a view, so it costs almost 0 memory! (compared to .repeat() which changes the tensor)

# Q-nets take **a lot** of memory.

Instead of creating all of them and then doing the similarity calculation, we will create batches and calculate similarities for just those q-nets, then discard those q-nets and move on to the next batch.

In [ ]:
def batch_encode_queries(encoder, queries, passage_embeddings):
  batch_size = 8
  similarity_scores = []

  num_queries = queries["input_ids"].shape[0]

  with torch.no_grad():
    for i in tqdm(range(0, num_queries, batch_size), desc="Creating q-nets and calculating similarity scores"):

        # create q-nets
        with autocast("cuda"):
          q_nets = encoder(
              input_ids=queries["input_ids"][i:i + batch_size].to(device),
              attention_mask=queries["attention_mask"][i:i + batch_size].to(device)
            ).representation


        passages_gpu = passage_embeddings.to(device)

        # Note: we use q_nets.num_queries (our repo's noTorch equivalent of q_nets.shape[0]) instead of batch_size
        # because the total number might not be divisible by batch_size so the last batch might be smaller than the actual batch size
        passages_batch = passages_gpu.unsqueeze(0).expand(q_nets.num_queries, -1, -1)

        # calculate similarity
        batch_scores = q_nets(passages_batch)
        similarity_scores.append(batch_scores.detach().cpu())


  scores_tensor = torch.cat(similarity_scores, dim=0)
  return scores_tensor


In [ ]:
similarity_scores = batch_encode_queries(query_encoder, query_inputs, passage_embeddings)

In [ ]:
# Case 1 - comparing a query to its respective passage

# In the simple case where each q_net only takes one passage, we can just
# reshape the passage_embeddings to (N, 1, H).
# passage_embeddings_single = passage_embeddings.unsqueeze(1)
# print(f"passage_embeddings shape: {passage_embeddings_single.shape}")
# giving the nueral network the input of passage_embeddings
# the output provides the relevance score of query 1 against passage 1, query 2 against passage 2, etc...
# scores = q_nets(passage_embeddings_single)
# print(f"scores: {scores}")

In [ ]:
# Case 2 - comparing a query to all passages

# The case where each q_net takes multiple passages
# meaning multiple passages are now associated with each of the queries

# this operation creates a 3D tensor which takes too much memory
# passage_embeddings_multi = passage_embeddings.repeat(N, 1).reshape(N, M, H)
# print(f"passage_embeddings shape: {passage_embeddings_multi.shape}")


# unbatched similarity scores for q-nets
# similarity_scores = q_nets(passage_embeddings_multi)
# print(f"similarity_scores shape: {similarity_scores.shape}")
#print(f"similarity_scores: {similarity_scores}")


In [ ]:
similarity_scores

## Create a dataset of Hard Negatives based on "Negative Hard Mining"
We take the top "incorrect" item similarities of each query as negatives

In [ ]:
"""
Desired format for each line in the JSONL file:
{
  "query": {
    "id": query ID,
    "content": query text,
  },
  "items": [
    {
      "id": passage ID,
      "content": passage text,
      "score": Optional teacher score,
      "type": Sometimes used to specify type of item,
    },
    {
        # another item
    },
  ]
}

Contrastive Loss with Hard Negatives: The positive must be the first item, all following items
will be treated as negative
"""

In [ ]:
from pathlib import Path
import json
import torch

def write_hardneg_contrastive_jsonl(
    pairs,                           
    similarity_scores: torch.Tensor,
    output_jsonl_path: str | Path,
    num_negatives: int = 8,
):
    """
    For each query:
      - items[0] = the correct entity
      - items[1:1+num_negatives] = top scoring incorrect entities
    """
    
    output_jsonl_path = Path(output_jsonl_path)
    output_jsonl_path.parent.mkdir(parents=True, exist_ok=True)

    # Ensure CPU tensor for safe indexing
    similarity_scores = similarity_scores.detach().cpu()


    # check shapes are correct
    N, M = similarity_scores.shape
    if len(pairs["mention"]) != N:
        raise ValueError(f"Mismatch: scores have N={N}, but pairs has {len(pairs['mention'])} mentions")
    if len(pairs["entity"]) != M:
        raise ValueError(f"Mismatch: scores have M={M}, but pairs has {len(pairs['entity'])} entities")


    # generate the jsonl entires
    with output_jsonl_path.open("w", encoding="utf-8") as f:
        for i in range(N):
            q_id = pairs["id"][i]
            mention = pairs["mention"][i]

            # Positive
            pos_item = {
                "id": pairs["id"][i],
                "content": pairs["entity"][i],
                "score": None,
                "type": None,
            }

            # Get top candidates, then filter out the true positive index
            _, top_idxs = torch.topk(similarity_scores[i], k=num_negatives+1, largest=True, sorted=True)

            neg_items = []
            for j in top_idxs.tolist():
                if j == i:
                    continue
                neg_items.append(
                    {
                        "id": pairs["id"][j],
                        "content": pairs["entity"][j],
                        "score": None,
                        "type": None,
                    }
                )
                if len(neg_items) >= num_negatives:
                    break

            if len(neg_items) < num_negatives:
                raise RuntimeError(f"Only collected {len(neg_items)} negatives for query {i} (M={M}).")

            entry = {
                "query": {"id": q_id, "content": mention},
                "items": [pos_item, *neg_items],
            }
            json.dump(entry, f, ensure_ascii=False)
            f.write("\n")

    print(f"Wrote {N} lines to {output_jsonl_path}")

In [ ]:
# generate jsonl for contrastiev loss for train split
write_hardneg_contrastive_jsonl(
    pairs=train_pairs,
    similarity_scores=similarity_scores,
    output_jsonl_path="bc5cdr_train_hypencoder_contrastive.jsonl",
    num_negatives=8,
)